### Introduction
In this notebook I use the PuLP package to find the most calorie efficient Mediterranean diet. Or in other words, the foods that provide the most nutrients with the least amount of calories in an attempt to satisfy all nutritional needs.

Import the usual suspects pandas, numpy, sqlite and PuLP a package used to solve linear problems.

In [82]:
#Imports
import pandas as pd
import numpy as np
import sqlite3 as db
import csv
#Used for linear optimization 
from pulp import *

### Getting USDA Food Nutrition Data
Connecting to the db 'nutrients.db' that I saved locally from link:

In [83]:
#Connect to the db
conn = db.connect('nutrients.db')

In [84]:
# Create a 'cursor' to execute commands
c = conn.cursor()

In [85]:
#Convert the table 'nutrients' to a pandas df named usda
dat = db.connect('nutrients.db')
query = dat.execute("SELECT * From nut_data")
cols = [column[0] for column in query.description]
usda = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

## Pulling list of foods and RDI values
Here we are converting two csv files into pandas dataframes. The first is a list of 100 foods that are part of the mediterranean diet. The second is a list of micronutrients that we want our solution to optimize for.

In [86]:
med_foods = pd.read_csv('Med_Diet_Food.csv', converters={'ID Number': lambda x: str(x)})
rdi_nutrients = pd.read_csv("RDI_Nutrients.csv")

Next we want to take only the Mediterranean food subset of the usda database

In [87]:
foods_list = med_foods['ID Number'].tolist()
usda_med_foods = usda.loc[usda['NDB_No'].isin(foods_list)]

Next we will convert the food Id Numbers and Nutrition Numbers to strings to keep the type consistent throughout the three tables

In [88]:
#Convert the Nutr_No to a string
usda_med_foods['Nutr_No'] = usda_med_foods['Nutr_No'].astype('string')
#Convert ID Number to a string although the lambda function from above does the same
med_foods['ID Number'] = med_foods['ID Number'].astype('string')
#Convert the Nutr_No in the rdi_nutrients table to a string
rdi_nutrients['Nutr_No'] = rdi_nutrients['Nutr_No'].astype('string')

/var/folders/xc/sqdfq69964zgmn2123jcdf0h0000gn/T/ipykernel_1426/2238444233.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usda_med_foods['Nutr_No'] = usda_med_foods['Nutr_No'].astype('string')


Our end goal is to have a df of the list of mediterranean foods with their micronutrient amounts (per 100g). To do this, we need to add a column for each micronutrient to each food in our list.

In [89]:
#Add the nutrients as columns to med_foods df
nutr_list = rdi_nutrients['Nutr_No'] 
for i in nutr_list:
    med_foods[i] = ''

These next steps are optional, but when I first tried to solve this problem the solution contained unrealistic amounts of red wine vinegar and certain spices. Although, it was surprising to find out how nutritious and low calorie these items are, most people will find it difficult to eat these items in large quantities.

In [90]:
#drop balsamic and red wine vinegar
med_foods = med_foods.drop([2,3],axis=0)

In [91]:
#Drop Duplicate Broccoli
med_foods = med_foods.drop([13],axis=0)

In [92]:
#Remove beef
med_foods = med_foods.drop([54],axis=0)

In [93]:
#Need to remove all of the spices (indices 83 to 99) because these are hard to eat in large quantities
med_foods.drop(med_foods.loc[83:99].index, inplace=True)

In [94]:
#Remove spices
med_foods.drop(med_foods.loc[80:81].index, inplace=True)

In [95]:
#Reset index and add new index
med_foods=med_foods.reset_index()
med_foods = med_foods.drop(['index'],axis=1)

Now we will add the nutrition values for each food left in our list of foods.

In [96]:
#For each row in med_foods, add the nutrient values for each column
for i in med_foods['ID Number']:
    for j in rdi_nutrients['Nutr_No']:
        if usda_med_foods.loc[(usda_med_foods.NDB_No == i) & (usda_med_foods.Nutr_No == j)]['Nutr_Val'].empty:
            val = np.NaN
        else:
            val = float(usda_med_foods.loc[(usda_med_foods.NDB_No == i) & (usda_med_foods.Nutr_No == j)]['Nutr_Val'])
        med_foods.loc[med_foods['ID Number'] == i,j] = val

In [98]:
#fill the NaN with 0.0
med_foods = med_foods.fillna(0.0)

### Linear Programming 
Now we can begin using the table that we created med_foods to solve our optimization problem

In [101]:
#Create an LP problem with LpProblem method in PuLP
prob = LpProblem("Simple Diet Problem",LpMinimize)

/Users/virginiasahagun/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [102]:
#Create dictionaries needed for problem
food_items = list(med_foods['Food'])

#Dict for Calories for all foods Calories = 208
calories = dict(zip(food_items,med_foods['208']))

#Dict for Protein
protein = dict(zip(food_items,med_foods['203']))

#Dict for carbs
carbs = dict(zip(food_items,med_foods['205']))

#Dict for fat
fat = dict(zip(food_items,med_foods['204']))

#Dict for Vit A
vit_A = dict(zip(food_items,med_foods['318']))

#Dict for Vit C
vit_C = dict(zip(food_items,med_foods['401']))

#Calcium
Calcium = dict(zip(food_items,med_foods['301']))

#Iron
Iron = dict(zip(food_items,med_foods['303']))

#Vit E
vit_E = dict(zip(food_items,med_foods['323']))

#Vit K
vit_K = dict(zip(food_items,med_foods['430']))

#thiamine
thiamine = dict(zip(food_items,med_foods['404']))

#riboflavin
riboflavin = dict(zip(food_items,med_foods['405']))

#niacin
niacin = dict(zip(food_items,med_foods['406']))

#vit_b6
vit_b6 = dict(zip(food_items,med_foods['415']))

#folate
folate = dict(zip(food_items,med_foods['417']))

#vit_b12
vit_b12 = dict(zip(food_items,med_foods['418']))

#pantothenic_acid 
pantothenic_acid = dict(zip(food_items,med_foods['410']))

#phosphorus 
phosphorus = dict(zip(food_items,med_foods['305']))

#magnesium
magnesium = dict(zip(food_items,med_foods['304']))

#zinc
zinc = dict(zip(food_items,med_foods['309']))

#selenium
selenium = dict(zip(food_items,med_foods['317']))

#copper
copper = dict(zip(food_items,med_foods['312']))

#manganese
manganese = dict(zip(food_items,med_foods['315']))

#potassium
potassium = dict(zip(food_items,med_foods['306']))

#choline
choline = dict(zip(food_items,med_foods['421']))

#dha
dha = dict(zip(food_items,med_foods['621']))

#epa
epa = dict(zip(food_items,med_foods['629']))

#ala
ala = dict(zip(food_items,med_foods['851']))

In [103]:
food_vars = LpVariable.dicts("Food",food_items,lowBound=0,cat='Continuous')

In [104]:
prob += lpSum([calories[i]*food_vars[i] for i in food_items])

In [105]:
# Calories
#prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 800.0, "calorieMinimum"
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 2000.0, "calorieMaximum"

# Carbs
#prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 20.0, "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 100.0, "CarbsMaximum"

# vit_A
prob += lpSum([vit_A[f] * food_vars[f] for f in food_items]) >= 3000.0, "vit_AMinimum"

# vit_C
prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) >= 90.0, "vit_CMinimum"

# Calcium
prob += lpSum([Calcium[f] * food_vars[f] for f in food_items]) >= 1000.0, "CalciumMinimum"

# Iron
prob += lpSum([Iron[f] * food_vars[f] for f in food_items]) >= 18.0, "IronMinimum"

# vit_K
prob += lpSum([vit_K[f] * food_vars[f] for f in food_items]) >= 120.0, "vit_KMinimum"

# thiamine
prob += lpSum([thiamine[f] * food_vars[f] for f in food_items]) >= 1.2, "thiamineMinimum"

# riboflavin
prob += lpSum([riboflavin[f] * food_vars[f] for f in food_items]) >= 1.3, "riboflavinMinimum"

# niacin
prob += lpSum([niacin[f] * food_vars[f] for f in food_items]) >= 16.0, "niacinMinimum"

# vit_b6
prob += lpSum([vit_b6[f] * food_vars[f] for f in food_items]) >= 1.7, "vit_b6Minimum"
#prob += lpSum([vit_b6[f] * food_vars[f] for f in food_items]) <= 2.1, "vit_b6Maximum"

# folate
prob += lpSum([folate[f] * food_vars[f] for f in food_items]) >= 400.0, "folateMinimum"

# vit_b12
prob += lpSum([vit_b12[f] * food_vars[f] for f in food_items]) >= 3, "vit_b12Minimum"

# pantothenic_acid
prob += lpSum([pantothenic_acid[f] * food_vars[f] for f in food_items]) >= 5.0, "pantothenic_acidMinimum"

# phosphorus
prob += lpSum([phosphorus[f] * food_vars[f] for f in food_items]) >= 1250.0, "phosphorusMinimum"

# magnesium
prob += lpSum([magnesium[f] * food_vars[f] for f in food_items]) >= 420.0, "magnesiumMinimum"

# zinc
prob += lpSum([zinc[f] * food_vars[f] for f in food_items]) >= 11.0, "zincMinimum"

# selenium
prob += lpSum([selenium[f] * food_vars[f] for f in food_items]) >= 55.0, "seleniumMinimum"

# copper
prob += lpSum([copper[f] * food_vars[f] for f in food_items]) >= 0.9, "copperMinimum"

# manganese
prob += lpSum([manganese[f] * food_vars[f] for f in food_items]) >= 2.3, "manganeseMinimum"

# potassium
prob += lpSum([potassium[f] * food_vars[f] for f in food_items]) >= 4700, "potassiumMinimum"

# choline
prob += lpSum([choline[f] * food_vars[f] for f in food_items]) >= 550, "cholineMinimum"

# dha
prob += lpSum([dha[f] * food_vars[f] for f in food_items]) >= 0.5, "dhaMinimum"

# epa
prob += lpSum([epa[f] * food_vars[f] for f in food_items]) >= 0.5, "epaMinimum"


# ala
prob += lpSum([ala[f] * food_vars[f] for f in food_items]) >= 1.6, "alaMinimum"

In [106]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/virginiasahagun/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/xc/sqdfq69964zgmn2123jcdf0h0000gn/T/4ca3b4529be546b7885b9c3da996abe7-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/xc/sqdfq69964zgmn2123jcdf0h0000gn/T/4ca3b4529be546b7885b9c3da996abe7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 1657 RHS
At line 1683 BOUNDS
At line 1684 ENDATA
Problem MODEL has 25 rows, 77 columns and 1549 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 25 (0) rows, 76 (-1) columns and 1547 (-2) elements
0  Obj 0 Primal inf 558.57783 (23)
10  Obj 751.25047
Optimal - objective value 751.25047
After Postsolve, objective 751.25047, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 751.2504661 - 10 iterations time 0.002, Pr

1

In [107]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [108]:
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Food_Anchovies = 0.6515848
Food_Arugula = 0.33524805
Food_Beet_Greens = 2.9796469
Food_Clams = 0.020585516
Food_Eggs = 0.51434696
Food_Zucchini = 26.229508


What does this mean in human? Since the usda serving size is 100g for all foods, we simply need to multiply these values by 100, to calculate how many grams of each item we would need to consume. For example, 0.65 x 100 equals 65g of anchovies, or a little less than 2.5 oz. 

### Conclusion
This solution starts off reasonable enough, but once you get to the last line you realize that it requires us to consume 2600 grams of zucchini per day. That is equivalent to 26 small or 15 medium sized zucchinis, a daily feat that would likely be unrealistic for most people. The zucchini notwithstanding, this solution does a really great job of providing most micronutrients (see the Vit_Min_Cronometer Screenshot file), and it accomplishes this with only 655 calories for all items. This leaves enough room for the 4 tablespoons of olive oil a day that are consumed in the Mediterranean, and that may provide many of the health benenfits but that do not provide many other micronutrients beyond Vitimin E. The only two nutrients that are not covered by our solution are Vitamin D (which I omitted because of how difficult it is to get from diet alone) and Vitamin B12. However, upping the serving of clams from 2g a day to 8g bridges the Vitamin B12 gap at a minimal calorie cost of about 9 calories. These tweaks, including the olive oil, bring the total calorie count to 1132 calories a day leaving enough room to add more variety, whatever form that might take.